# MSSQL DB Connection Template  
### using pyodbc library


### Installing Libraries

In [ ]:
#installing packages
!pip install pyodbc pandas seaborn  matplotlib numpy xgboost scikit-learn

### Importing Libraries

In [ ]:
#importing libraries
import pyodbc
import pandas as pd

### Connecting to Database

In [53]:
# Define connection parameters
server = 'rebates-mssql-2019-2-dev.con8g81q8rff.us-east-1.rds.amazonaws.com'  # e.g., 'your-db-name.region.rds.amazonaws.com'
database = 'dev-rebate-merit'
username = 'jamalif'
password = 'eUtYIwguuC'
driver = '{ODBC Driver 17 for SQL Server}'  # Make sure this driver is installed

In [54]:
# Create connection string
connection_string = f'''
    DRIVER={driver};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    Encrypt=no;
    TrustServerCertificate=yes;
    Connection Timeout=30;
'''

try:
    conn = pyodbc.connect(connection_string)
    print("✅ Connection successful!")
except Exception as e:
    print("❌ Connection failed:", e)

✅ Connection successful!


### Retrieving Data form Database

In [55]:
# Define your SQL query
query = '''

        WITH NLP_Sales AS 
            (
                SELECT sid.[orderdate] as [Date], SID.PayeeCustomerAccount As [Customer], SUM(SID.InvAmt) as [Sales]
                FROM SalesInvoiceDetail SID
                JOIN batch B ON SID.batchid = B.id 
                    AND B.schemeid IN (6918,6401)
                JOIN CustomerMaster CM on SID.PayeeCustomerAccount = CM.CustomerAccount
                    AND BatchNo IN (15)
                    AND ((b.BatchTypeId = 1 AND MonthlyPaymentApplicable = 1))
                --OR (b.BatchTypeId = 2 and MonthlyPaymentApplicable = 0))
                    AND SID.salesinvoicetypeid IN (1, 12)
                    AND excluded = 0 AND  category = 'Nobivac Vaccines'
                --WHERE SID.CustomerAccount = '0000008687'
                GROUP BY sid.[orderdate], SID.PayeeCustomerAccount 
                
            )

            SELECT * FROM NLP_Sales
            Order By [Date]

        '''

# Execute and store results in a pandas DataFrame
try:
    df = pd.read_sql(query, conn)
    print("✅ Query executed and data loaded into DataFrame.")
    display(df.head())  # Show first few rows
except Exception as e:
    print("❌ Failed to execute query:", e)


C:\Users\Fuad.Jamali\AppData\Local\Temp\ipykernel_15424\1241077748.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✅ Query executed and data loaded into DataFrame.


,Date,Customer,Sales
0,2023-01-01,0010077313,936.00
1,2023-01-01,0010279505,5796.25
2,2023-01-02,0010013607,464.75
3,2023-01-02,0010021818,4207.50
4,2023-01-02,0000008687,982.75


In [57]:
df.shape

(6586, 3)